<a href="https://colab.research.google.com/github/lilaceri/Working-with-data-/blob/main/Decision_Tree_coded_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding a simple decision tree
---

In this worksheet we are going to work with a data set, using the idea of a decision tree class.  We are going to simplify the model and use Python code to make a simple decision tree classification model.  We will do this for two reasons:
*   writing the code is often good for helping to understand what is going on under the bonnet of a library function
*   it is a good coding exercise for practice as it mostly depends on calculations and if..elif..else statements

In this worksheet we are going to code a decision tree which will use the calculated probabilities to make decisions about whether a row of given data would be classified as Iris-virginica, or not, based on sepal and petal dimensions.  It is easier to classify between two values (Iris-virginica or not).  Later, using this information, species would be further predicted by probabilities of error.

![Iris-petals and sepals](https://www.math.umd.edu/~petersd/666/html/iris_with_labels.jpg)

The workflow is:
*  divide the data set into 70% of the rows for training and 30% for testing  (we can increase the size of the train set later)
*  find the median for each of the 4 size columns
*  calculate the proportion of each column that are on or above median that are of a species (ie proportion of petal-lengths on or above median that are Iris-virginica)
*  infer the proportion of each that are not of that species (using 1 - proportion above).  In both cases we are looking to find if either of these is 1, which could be infered as definitely not that species. 
*  calculate a Gini Index that will indicate the probability that a prediction will be incorrect
*  use the results of the Gini Index to model a decision tree
*  code the decision tree model into a function that will return whether or not a row in the test set is predicted to be of species Iris-virginica
*  use the decision tree function to predict, for each row in the test set, if the species will be Iris-virginicia or not, using a set of nested if statements to classify
*  compare the predicted values against the actual values in the test set - what proportion were predicted correctly?


### Exercise 1 - investigate the iris data set
---
Let's start by looking at the data.  We are going to use a data set that contains data on iris flowers.

Read the data at this location: https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv into a dataframe called iris_data

The columns in the CSV file do not have headings, when you read the file, add column headings like this:
```
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'species']
iris_data = pd.read_csv(url, names=names)
```
*  Take a look at the column info (how many columns, what type of data, any missing data?)
*  Take a look at the data values in the first 10 and the last 10 records to get an idea of the type of values included
*  Find out how many unique values there are in the species column
*  Find out the maximum, minimum, median and upper and lower quartile values in each of the columns


In [153]:
import pandas as pd 
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'species']
iris_data = pd.read_csv(url, names=names)

# iris_data.info()
# iris_data.head(10)
# iris_data.tail(10)
iris_data['species'].unique()
# max = iris_data.max()
# min = iris_data.min()
# med = iris_data.median()
# upper = iris_data.quantile(0.75)
# lower = iris_data.quantile(0.25)
# print('max', max, 'min', min, 'med', med, 'upper', upper,'lower', lower)

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

### Exercise 2 - split the data into train and test sets
---

Split the data set into and 70% train, 30% test, split.  Run the cell below. Add code to inspect the `train` data set.


In [154]:
# import the train_test_split function
from sklearn.model_selection import train_test_split

# create the classification variables from the all columns
train, test = train_test_split(iris_data, test_size=0.30, random_state=1)

test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 14 to 58
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal-length  45 non-null     float64
 1   sepal-width   45 non-null     float64
 2   petal-length  45 non-null     float64
 3   petal-width   45 non-null     float64
 4   species       45 non-null     object 
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


### Exercise 3 - assumptions
---

Let's make some assumptions based on the data

1.  Iris-setosa, Iris-versicolor, Iris-virginica are the full range of types of iris to be analysed
2.  Although this is a small data set, the medians are fairly representative

With these in mind, let's start by classifying sepal/petal size into long/short and wide/narrow with values on or above the median taken as long or wide and those below as short or narrow.

This is a starting point.  We will be trying to find a value (indicator) for each column where rows on or above do not contain any of a particular species, this might indicate that this column is a good (if not rough) indicator of species.

Calculate, and store the medians of the four columns

**Test**:
Display train.describe() to see the value of the medians of the training set
Print the four medians and compare to the output of train.describe() to check that they have been calculated correctly

In [155]:
sl_indicator = iris_data['sepal-length'].quantile(.1)
sw_indicator = iris_data['sepal-width'].quantile(.05)
pl_indicator = iris_data['petal-length'].quantile(.1)
pw_indicator = iris_data['petal-width'].quantile(.1)

print(sl_indicator, sw_indicator, pl_indicator, pw_indicator)
display(train.describe())

4.8 2.3449999999999998 1.4 0.2


,sepal-length,sepal-width,petal-length,petal-width
count,105.000000,105.000000,105.000000,105.000000
mean,5.800000,3.033333,3.739048,1.189524
std,0.844553,0.415254,1.787429,0.784333
min,4.300000,2.000000,1.000000,0.100000
25%,5.000000,2.800000,1.600000,0.200000
50%,5.800000,3.000000,4.200000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.700000,4.400000,6.900000,2.500000


### Exercise 4 - Calculate the proportion of values on or above the indicator that are of species `Iris-virginica`

We are going to focus on the `Iris-virginica` species.

First we will calculate, for each dimension column (`sepal-length, sepal-width, petal-length, petal-width`) what proportion of values in that column, where the value is on or above the median, are classified as `Iris-virginica`.

We will do this by filtering all the records in each column of the the `train` set that are on or above the median and match the species .  Then use the outcome to calculate the proportion of the full `train` set for which a value on or above the median that are of species `Iris-virginica`.

*  filter for values in the `sepal-length` column being on or above the median (`sl_indicator`) and the species column being `Iris-virginica`.  Then divide the count of rows in this filtered dataset by the count of rows in a second data set, filtered for just the value being on or above the median.

*  Do this for all four columns, for `Iris-virginica`  (4 operations).

Print the results to see which columns look like they might predict the species as `Iris-virginica` (the result is 1).  The highest numbers may be the most, but we will do some more before coming to this conclusion.

*  By definition, those on or above the median that are NOT Iris_virginica will be `1 - the proportion of those that are.  Calculate these

The first one has been done for you.

*  We will also need the proportion of those BELOW the median that are NOT Iris-virginica.  Calculate these in the same way



In [156]:
# calculate the proportion of results where the value is on or above median that are of the species Iris-virginica
sl_vi_above = (train[(train['sepal-length'] >= sl_indicator) & (train['species'] == 'Iris-versicolor')]['sepal-length'].count()) / (train[train['sepal-length'] >= sl_indicator]['sepal-length'].count())
sw_vi_above = (train[(train['sepal-width'] >= sw_indicator) & (train['species'] == 'Iris-versicolor')]['sepal-width'].count()) / (train[train['sepal-width'] >= sw_indicator]['sepal-width'].count())
pl_vi_above = (train[(train['petal-length'] >= pl_indicator) & (train['species'] == 'Iris-versicolor')]['petal-length'].count()) / (train[train['petal-length'] >= pl_indicator]['petal-length'].count())
pw_vi_above = (train[(train['petal-width'] >= pw_indicator) & (train['species'] == 'Iris-versicolor')]['petal-width'].count()) / (train[train['petal-width'] >= pw_indicator]['petal-width'].count())

print(sl_vi_above, sw_vi_above, pl_vi_above, pw_vi_above)

# calculate the proportion of results where the column is above median that are NOT of the species Iris-virginica

no_sl_vi_above = (train[(train['sepal-length'] >= sl_indicator) & (train['species'] != 'Iris-versicolor')]['sepal-length'].count()) / (train[train['sepal-length'] >= sl_indicator]['sepal-length'].count())
no_sw_vi_above = (train[(train['sepal-width'] >= sw_indicator) & (train['species'] != 'Iris-versicolor')]['sepal-width'].count()) / (train[train['sepal-width'] >= sw_indicator]['sepal-width'].count())
no_pl_vi_above = (train[(train['petal-length'] >= pl_indicator) & (train['species'] != 'Iris-versicolor')]['petal-length'].count()) / (train[train['petal-length'] >= pl_indicator]['petal-length'].count())
no_pw_vi_above = (train[(train['petal-width'] >= pw_indicator) & (train['species'] != 'Iris-versicolor')]['petal-width'].count()) / (train[train['petal-width'] >= pw_indicator]['petal-width'].count())

print(no_sl_vi_above, no_sw_vi_above, no_pl_vi_above, no_pw_vi_above)

0.3333333333333333 0.2727272727272727 0.32323232323232326 0.32323232323232326
0.6666666666666666 0.7272727272727273 0.6767676767676768 0.6767676767676768


### Exercise 5 - Calculate the proportion of each column where the value is below median that are of species `Iris-virginica`

Repeat the code above, this time looking for values below the mean

In [157]:
# calculate the proportion of results where the value is below median that are of the species Iris-virginica

sl_vi_below = train[(train['sepal-length'] < sl_indicator) & (train['species'] == 'Iris-versicolor')]['sepal-length'].count() / train[train['sepal-length'] < sl_indicator]['sepal-length'].count()
sw_vi_below = (train[(train['sepal-width'] < sw_indicator) & (train['species'] == 'Iris-versicolor')]['sepal-width'].count()) / (train[train['sepal-width'] < sw_indicator]['sepal-width'].count())
pl_vi_below = (train[(train['petal-length'] < pl_indicator) & (train['species'] == 'Iris-versicolor')]['petal-length'].count()) / (train[train['petal-length'] < pl_indicator]['petal-length'].count())
pw_vi_below = (train[(train['petal-width'] < pw_indicator) & (train['species'] == 'Iris-versicolor')]['petal-width'].count()) / (train[train['petal-width'] < pw_indicator]['petal-width'].count())


print(sl_vi_below, sw_vi_below, pl_vi_below, pw_vi_below)

# calculate the proportion of results where the column is below median that are NOT of the species Iris-virginica

no_sl_vi_below = 1-sl_vi_below
no_sw_vi_below = 1-sw_vi_below
no_pl_vi_below = 1-pl_vi_below
no_pw_vi_below = 1-pw_vi_below

print(no_sl_vi_below, no_sw_vi_below, no_pl_vi_below, no_pw_vi_below)

0.0 0.8333333333333334 0.0 0.0
1.0 0.16666666666666663 1.0 1.0


### Exercise 5 - calculate Gini index for `Iris-virginica`
---

Each time you split the data set into `train` and `test`, you will get a slightly different mix and so your `train` data set will be slightly different.  We are going to try to look at how well we might predict a particular species from the 3 columns.   Let's use the `Iris-virginica` species and try to predict if a row would be that species or not, based on the four dimensions columns.

A Gini Index is a measure of the probability of a randomly chosen prediction being incorrect.  The most influential column will have the lowest Gini Index and that will be put at the top of our decision tree.

The formula for the Gini Index is:

*Gini Index = 1 - (the sum of the squares of the proportion values calculated above)*

To calculate the Gini Index for , use the following example:

`gini_sl_vi = 1 - (sl_vi_above**2 + no_sl_vi_above**2)`

The first one has been done for you.






In [158]:
# calculate the Gini Index for the proportion of those below median which are Iris_virginica, for all four columns

gini_sl_vi_above = 1 - (sl_vi_above**2 + no_sl_vi_above**2)
gini_sw_vi_above = 1 - (sw_vi_above**2 + no_sw_vi_above**2)
gini_pl_vi_above = 1 - (pl_vi_above**2 + no_pl_vi_above**2)
gini_pw_vi_above = 1 - (pw_vi_above**2 + no_pw_vi_above**2)

print(gini_sl_vi_above, gini_sw_vi_above, gini_pl_vi_above, gini_pw_vi_above)

# calculate the Gini Index for the proportion of those below median which are Iris_virginica, for all four columns

gini_sl_vi_below = 1 - (sl_vi_below**2 + no_sl_vi_below**2)
gini_sw_vi_below = 1 - (sw_vi_below**2 + no_sw_vi_below**2)
gini_pl_vi_below = 1 - (pl_vi_below**2 + no_pl_vi_below**2)
gini_pw_vi_below = 1 - (pw_vi_below**2 + no_pw_vi_below**2)

print(gini_sl_vi_below, gini_sw_vi_below, gini_pl_vi_below, gini_pw_vi_below)


0.4444444444444444 0.39669421487603307 0.43750637690031624 0.43750637690031624
0.0 0.2777777777777777 0.0 0.0


### Exercise 6 - add weights to the index
---

Lastly, we are going to weight the calculation by applying the proportion of those that are and aren't `Iris_virginica`  

This is the calculation for the sepal-length column:
1.  Use the proportion of values in **whole sepal-length column** that are on or above median:  
`sl_vi_above_indicator = train[train['sepal-length'] >= sl_indicator]['sepal-length'].count() / train['sepal-length'].count()`

2.  Do the same to calculate the proportion of values below the median

3.  Calculate weightings using the formula:  
`weighted_gini_sl_vi = sl_vi_above_indicator * gini_sl_vi_above + sl_vi_below_indicator * gini_sl_vi_below`

Do this for each of the four columns

In [159]:
# calculate the proportion of values in sepal-length column that are on or above mean, then calculate the weighted Gini Index

sl_vi_above_indicator = train[train['sepal-length'] >= sl_indicator]['sepal-length'].count() / train['sepal-length'].count()
sl_vi_below_indicator = train[train['sepal-length'] < sl_indicator]['sepal-length'].count() / train['sepal-length'].count()
weighted_gini_sl_vi = sl_vi_above_indicator * gini_sl_vi_above + sl_vi_below_indicator * gini_sl_vi_below
print(weighted_gini_sl_vi)

# calculate the weighted Gini Index for sepal-width

sw_vi_above_indicator = train[train['sepal-width'] >= sw_indicator]['sepal-width'].count() / train['sepal-width'].count()
sw_vi_below_indicator = train[train['sepal-width'] < sw_indicator]['sepal-width'].count() / train['sepal-width'].count()
weighted_gini_sw_vi = sw_vi_above_indicator * gini_sw_vi_above + sw_vi_below_indicator * gini_sw_vi_below
print(weighted_gini_sw_vi)

# calculate the weighted Gini Index for petal_length

pl_vi_above_indicator = train[train['petal-length'] >= pl_indicator]['petal-length'].count() / train['petal-length'].count()
pl_vi_below_indicator = train[train['petal-length'] < pl_indicator]['petal-length'].count() / train['petal-length'].count()
weighted_gini_pl_vi = pl_vi_above_indicator * gini_pl_vi_above + pl_vi_below_indicator * gini_pl_vi_below
print(weighted_gini_pl_vi)

# calculate the weighted Gini Index for petal-width

pw_vi_above_indicator = train[train['petal-width'] >= pw_indicator]['petal-width'].count() / train['petal-width'].count()
pw_vi_below_indicator = train[train['petal-width'] < pw_indicator]['petal-width'].count() / train['petal-width'].count()
weighted_gini_pw_vi = pw_vi_above_indicator * gini_pw_vi_above + pw_vi_below_indicator * gini_pw_vi_below
print(weighted_gini_pw_vi)

0.4063492063492063
0.3898989898989899
0.41250601250601243
0.41250601250601243


### Exercise 6 - Make a decision tree
---

Use pencil and paper or a graphical application to create a decision tree using the following rules (**use the picture below as a guide only - yours will be different**):

*  the column with a 0.0 initial Gini Index and the lowest weighted Gini Index is placed at the top
*  other columns with a 0.0 initial Gini Index are placed in order below
*  the rest of the columns are placed in order below these

Any column where one branch (on or above median OR below median) has an initial Gini Index of 0, could be classified as a strong indicator of Iris_virginica being the species.  Anything else doesn't have enough certainty.

Let's code the decision tree using the following logic for this decision tree (yours might be slightly different):

![Decision tree](https://drive.google.com/uc?id=1CTo23EHwR2IPCRjcfSyCQsT_oQ5Exwso)

In the decision tree above, there is no certainty below petal-length so our decision tree will only include petal-width and petal-length.




In [162]:
def predict_species(df):
  # ADD CODE HERE TO RETURN None if petal-width is below pw_decision_line or if petal-length is below pl_decision_line, otherwise return 'Iris-virginia'
  if df['petal-length'] >= pl_indicator:
    if df['petal-width'] >= pw_indicator:
      if df['sepal-length'] >= sl_indicator:
        # if df['sepal-width'] >= sw_indicator:
        return "Iris-versicolor"
        # else:
        #   return "not Iris-versicolor"
      else:
        return "not Iris-versicolor"
    else:
      return "not Iris-versicolor"
  else:
    return "not Iris-versicolor"

# use the get_species(df) function to predict the species, count how many are predicted correct and use this to calculate the proportion correct
correct = 0
test_size = test.shape[0]
for i in range(0, test_size):
  species = predict_species(test.iloc[i])
  if species == test.iloc[i]['species']:
      correct += 1

print ("Proportion correctly identified", correct / test_size) 


Proportion correctly identified 0.4


### Exercise 7 - expand the training set
---

Let's make the training set a bit bigger so that there are more rows to use in the analysis.

*  go back and change the train/test proportions to 80% and 20%
*  save the notebook and run all the code cells again

# SUM UP what you have found out so far
---
write your summary here

### Exercise 8 - change the measure for sepal-length

We are currently using the median to act as the indicator line for all 4 columns.  This is helping us to identify petal-length and petal-width as good indicators around the median.  We can use the decision tree with different indicators.

Change the `sepal-length` indicator so that you are instead using the lower quartile.  The code should not need changing except for where you calculated the value for `sl_indicator`.

Run all the code again.  Is the proportion of correct values better or worse this time?   Is the decision tree still appropriate?


WHAT DO YOU NOTICE? (write your answer here)
---




### Exercise 9 - try different indicators for sepal-length
---

Do the same again trying different indicators.  If upper or lower quartiles don't help, try using another percentile (e.g. .quantile(0.2). Is it making any difference?  What indicators give the best looking results?

WHAT DO YOU NOTICE? 

write your answer here



### Exercise 10 - try a different species

Run the median test again for the Iris-versicolor species.  Again, try some different indicators.

What are the results.  Record them in the text cell below:

RESULTS FOR Iris-versicolor
---  
write your findings here

# New logic introduced in this worksheet:

1.  Adding headings to a CSV if none currently exist
2.  Splitting a data set into train and test sets

# Reflection
----

## What skills have you demonstrated in completing this notebook?

Your answer: 

## What caused you the most difficulty?

Your answer: 